# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Ames Housing Data once more

In [1]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

In [4]:
preprocessed.head()
predictors = preprocessed.drop(columns=['SalePrice_log'])
predictors

,LotArea_log,1stFlrSF_log,GrLivArea_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,-0.133185,-0.803295,0.529078,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.113403,0.418442,-0.381715,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0.419917,-0.576363,0.659449,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0.103311,-0.439137,0.541326,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.878108,0.112229,1.281751,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.259100,-0.465447,0.416538,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1456,0.725171,1.980456,1.106213,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1457,-0.002324,0.228260,1.469438,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1458,0.136814,-0.077546,-0.854179,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Perform stepwise selection

The function for stepwise selection is copied below. Use this provided function on your preprocessed Ames Housing data.

In [2]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [19]:
# Your code here
selected_cols = stepwise_selection(predictors, ames['SalePrice'], verbose=True)
selected_cols

Add  GrLivArea_log                  with p-value 3.136e-211
Add  KitchenQual_TA                 with p-value 1.23896e-57
Add  1stFlrSF_log                   with p-value 1.17581e-42
Add  Neighborhood_NridgHt           with p-value 2.62524e-34
Add  Neighborhood_NoRidge           with p-value 1.23851e-23
Add  SaleType_New                   with p-value 5.87745e-20
Add  Neighborhood_StoneBr           with p-value 7.54973e-17
Add  LotArea_log                    with p-value 4.37652e-14
Add  KitchenQual_Fa                 with p-value 1.00957e-15
Add  KitchenQual_Gd                 with p-value 3.38037e-32
Add  Neighborhood_OldTown           with p-value 7.01589e-12
Add  Neighborhood_Edwards           with p-value 2.11781e-10
Add  BldgType_Duplex                with p-value 4.39178e-08
Add  Neighborhood_Somerst           with p-value 2.49671e-07
Add  Neighborhood_NAmes             with p-value 4.16183e-06
Add  Neighborhood_IDOTRR            with p-value 7.08692e-07
Add  Neighborhood_SWISU  

['GrLivArea_log',
 'KitchenQual_TA',
 '1stFlrSF_log',
 'Neighborhood_NridgHt',
 'Neighborhood_NoRidge',
 'SaleType_New',
 'Neighborhood_StoneBr',
 'LotArea_log',
 'KitchenQual_Fa',
 'KitchenQual_Gd',
 'Neighborhood_OldTown',
 'Neighborhood_Edwards',
 'BldgType_Duplex',
 'Neighborhood_Somerst',
 'Neighborhood_NAmes',
 'Neighborhood_IDOTRR',
 'Neighborhood_SWISU',
 'Neighborhood_Sawyer',
 'Neighborhood_BrkSide',
 'Neighborhood_NWAmes']

### Build the final model again in Statsmodels

In [22]:
final_pd = pd.concat([preprocessed[selected_cols], ames['SalePrice']],axis=1)
ols_pd = final_pd.rename(columns={'1stFlrSF_log':'FirstFlrSF_log'})
ols_pd.head()

,GrLivArea_log,KitchenQual_TA,FirstFlrSF_log,Neighborhood_NridgHt,Neighborhood_NoRidge,SaleType_New,Neighborhood_StoneBr,LotArea_log,KitchenQual_Fa,KitchenQual_Gd,...,Neighborhood_Edwards,BldgType_Duplex,Neighborhood_Somerst,Neighborhood_NAmes,Neighborhood_IDOTRR,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_BrkSide,Neighborhood_NWAmes,SalePrice
0,0.529078,0,-0.803295,0,0,0,0,-0.133185,0,1,...,0,0,0,0,0,0,0,0,0,208500
1,-0.381715,1,0.418442,0,0,0,0,0.113403,0,0,...,0,0,0,0,0,0,0,0,0,181500
2,0.659449,0,-0.576363,0,0,0,0,0.419917,0,1,...,0,0,0,0,0,0,0,0,0,223500
3,0.541326,0,-0.439137,0,0,0,0,0.103311,0,1,...,0,0,0,0,0,0,0,0,0,140000
4,1.281751,0,0.112229,0,1,0,0,0.878108,0,1,...,0,0,0,0,0,0,0,0,0,250000


In [25]:
# Your code here
from statsmodels.formula.api import ols
ols_predictors = list(ols_pd.columns[1:-2])
selected_sum = '+'.join(ols_predictors)
formula = f"SalePrice ~ {selected_sum}"
model = ols(formula=formula, data=ols_pd).fit()

In [26]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     231.4
Date:                Sun, 23 Aug 2020   Prob (F-statistic):               0.00
Time:                        15:38:58   Log-Likelihood:                -17552.
No. Observations:                1460   AIC:                         3.514e+04
Df Residuals:                    1441   BIC:                         3.524e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             2.638e+05   5063.676     52.104      0.000    2.54e+05    2.74e+05
KitchenQual_TA       -9.419e+04   5318.417    -17.711      0.000   -1.05e+05   -8.38e+04
FirstFlrSF_log        1.913e+04   1358.485     14.079      0.000    1.65e+04    2.18e+04
Neighborhood_NridgHt  5.384e+04   5586.594      9.638      0.000    4.29e+04    6.48e+04
Neighborhood_NoRidge  1.042e+05   6698.219     15.557      0.000    9.11e+04    1.17e+05
SaleType_New          1.977e+04   4346.524      4.548      0.000    1.12e+04    2.83e+04
Neighborhood_StoneBr  6.515e+04   8556.982      7.614      0.000    4.84e+04    8.19e+04
LotArea_log           1.563e+04   1234.593     12.657      0.000    1.32e+04     1.8e+04
KitchenQual_Fa       -1.082e+05   8571.128    -12.629      0.000   -1.25e+05   -9.14e+04
KitchenQual_Gd         -6.6e+04   4903.548    -13.460      0.000   -7.56e+04   -5.64e+04
Neighborhood_OldTown  -3.57e+04   4278.530     -8.344      0.000   -4.41e+04   -2.73e+04
Neighborhood_Edwards -4.399e+04   4504.086     -9.767      0.000   -5.28e+04   -3.52e+04
BldgType_Duplex      -2.149e+04   5922.678     -3.629      0.000   -3.31e+04   -9872.585
Neighborhood_Somerst  2.472e+04   4992.794      4.952      0.000    1.49e+04    3.45e+04
Neighborhood_NAmes   -3.284e+04   3325.124     -9.875      0.000   -3.94e+04   -2.63e+04
Neighborhood_IDOTRR  -4.725e+04   7088.686     -6.666      0.000   -6.12e+04   -3.33e+04
Neighborhood_SWISU    -2.18e+04   8417.914     -2.589      0.010   -3.83e+04   -5284.948
Neighborhood_Sawyer  -3.669e+04   5117.822     -7.170      0.000   -4.67e+04   -2.67e+04
Neighborhood_BrkSide -2.696e+04   5825.952     -4.628      0.000   -3.84e+04   -1.55e+04
==============================================================================
Omnibus:                      466.168   Durbin-Watson:                   1.954
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5158.379
Skew:                           1.159   Prob(JB):                         0.00
Kurtosis:                      11.912   Cond. No.                         13.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [27]:
# Your code here
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
sk_predictors = ols_pd.drop(columns=['SalePrice'])
linereg = LinearRegression()
selector = RFE(linereg, n_features_to_select=5)
selector = selector.fit(sk_predictors, ols_pd['SalePrice'])

In [29]:
list(zip(sk_predictors,selector.support_))

[('GrLivArea_log', False),
 ('KitchenQual_TA', True),
 ('FirstFlrSF_log', False),
 ('Neighborhood_NridgHt', False),
 ('Neighborhood_NoRidge', True),
 ('SaleType_New', False),
 ('Neighborhood_StoneBr', True),
 ('LotArea_log', False),
 ('KitchenQual_Fa', True),
 ('KitchenQual_Gd', True),
 ('Neighborhood_OldTown', False),
 ('Neighborhood_Edwards', False),
 ('BldgType_Duplex', False),
 ('Neighborhood_Somerst', False),
 ('Neighborhood_NAmes', False),
 ('Neighborhood_IDOTRR', False),
 ('Neighborhood_SWISU', False),
 ('Neighborhood_Sawyer', False),
 ('Neighborhood_BrkSide', False),
 ('Neighborhood_NWAmes', False)]

Fit the linear regression model again using the 5 selected columns

In [34]:
# Your code here
sk_predictors_final = sk_predictors.loc[:,['KitchenQual_TA', 'Neighborhood_NoRidge', 'Neighborhood_StoneBr', 
                                    'KitchenQual_Fa', 'KitchenQual_Gd']]
sk_predictors_final
linereg.fit(sk_predictors_final, ols_pd['SalePrice'])
print(linereg.coef_)
print(linereg.intercept_)

[-177989.87487524  124306.58146361   66314.10558728 -212048.9321104
 -114733.19191543]
317614.13723860105


Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [36]:
# Your code here
yhat = linereg.predict(sk_predictors_final)
yhat

array([202880.94532317, 139624.26236336, 202880.94532317, ...,
       202880.94532317, 202880.94532317, 139624.26236336])

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [37]:
# Your code here
ss_res = sum((ols_pd['SalePrice'] - yhat)**2)
ss_tot = sum((ols_pd['SalePrice'] - ols_pd['SalePrice'].mean())**2)
r_squared = 1 - ss_res/ss_tot

r_squared_adj = 1 - (1 - r_squared)*((len(ols_pd['SalePrice'])-1)/(len(ols_pd['SalePrice'])-5-1))

print(r_squared)
print(r_squared_adj)
# r_squared is 0.239434  
# adjusted_r_squared is 0.236818

0.5306050470844421
0.5289908966273735


## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Ames Housing dataset! 